In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import pearsonr
import math
from scipy import spatial
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from math import sqrt
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [13]:
#confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
# confirmed = pd.read_csv('time_series_covid19_deaths_global.csv')
# recovered = pd.read_csv('time_series_covid19_recovered_global.csv')

# latest
confirmed = pd.read_csv('time_series_covid19_deaths_global.csv')
#confirmed= pd.read_csv('deaths_global_latest.csv')

# Util

In [14]:
def scorer(y_pred, y_test):
    return np.sum([(np.log10(x) - np.log10(y))**2 for x, y in zip(y_pred, y_test) if x>1 and y>100])

In [15]:
def evaluate(sorted_combined):
    n = len(sorted_combined[0][2])
    scores = []
    for day in range(n):
        y_real = []
        y_pred = []
        for a in sorted_combined:
            
            y_real = y_real + [a[2]['cases'].values[day]]
            y_pred = y_pred + [a[3]['Prediction'].values[day]]

        y_pred = [int(np.ceil(x)) for x in y_pred]
        sc = scorer(y_pred, y_real)
        print(day, sc)
        scores.append(sc)
    return scores, y_pred, y_real

In [16]:
def evaluate_ensemble(sorted_combined, sorted_combined1):
    n = len(sorted_combined[0][2])
    scores = []
    for day in range(n):
        y_real = []
        y_pred = []
        y_pred1 = []
        for i in range(len(sorted_combined)):
            a = sorted_combined[i]
            b = sorted_combined1[i]
            y_real = y_real + [a[2]['cases'].values[day]]
            y_pred = y_pred + [a[3]['Prediction'].values[day]]
            y_pred1 = y_pred1 + [b[3]['Prediction'].values[day]]

        y_pred_ensembled = [int(np.ceil((x+x1)/2.0) ) for x,x1 in zip(y_pred, y_pred1)]
        sc = scorer(y_pred_ensembled, y_real)
        print(day, sc)
        scores.append(sc)
    return scores, y_pred_ensembled,y_pred, y_pred1,  y_real

# Auto Arima

In [17]:
a=10
columns = confirmed.columns[4:]
df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:]})
df.index = pd.to_datetime(columns)
train = df
valid = df[-7:]
print(train['cases'][-2])

0


In [18]:
def my_arima_model(index, train, valid, do_plot=0):
    if train['cases'][-1]==train['cases'][-2] and train['cases'][-2]==train['cases'][-3]:
        forecast = [train['cases'][-1]]*(len(valid))
        print('......', forecast)
    else:
        model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True, information_criterion='bic')
        model.fit(train)
        forecast = model.predict(n_periods=len(valid))
        print('---------', forecast)

    forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])
    rms = sqrt(mean_squared_error(valid,forecast))
    print(index, rms)
    #plot the predictions for validation set
    if do_plot==1:
        plt.plot(train, label='Train')
        plt.plot(valid, label='Valid')
        plt.plot(forecast, label='Prediction')
        plt.legend()
        plt.show()
    return rms, forecast

In [19]:
combined = []
columns = confirmed.columns[4:]
for a in range(len(confirmed)):
    df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:]})
    df.index = pd.to_datetime(columns)
    train = df
    valid = df[-7:]
    rms, forecast = my_arima_model(a, train, valid)
    current = [rms, a, valid, forecast, train]
    combined.append(current)
    

Performing stepwise search to minimize bic
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=232.759, BIC=247.824, Time=0.341 seconds
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=305.239, BIC=310.261, Time=0.012 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=276.791, BIC=284.323, Time=0.029 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=241.948, BIC=249.481, Time=0.083 seconds
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=False); AIC=303.267, BIC=305.778, Time=0.012 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=247.125, BIC=259.679, Time=0.309 seconds
Near non-invertible roots for order (1, 2, 2)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 1.000)
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=239.628, BIC=252.183, Time=0.121 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=237.765, BIC=255.341, Time=0.498 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0,

In [9]:
score, y_pred, y_real = evaluate(combined)
print("Score ",score)
print("Prediction, Actual")
arima_pred_df = pd.DataFrame({"Actual": y_real, "Prediction":y_pred})
#arima_pred_df.to_csv("21_april_deaths_arima_pred_df.csv", index=False)
#arima_pred_df.head()

0 1.8274145718247206
1 1.6534582596724705
2 1.5507180237169553
3 1.4140231719620031
4 1.4407783841055748
5 1.486147128891213
6 1.3867621990016685
7 1.2368078427650222
Score  [1.8274145718247206, 1.6534582596724705, 1.5507180237169553, 1.4140231719620031, 1.4407783841055748, 1.486147128891213, 1.3867621990016685, 1.2368078427650222]
Prediction, Actual


# Holt winter

In [10]:
def my_arima_model(index, train, valid, do_plot=0):
    if train['cases'][-1]==train['cases'][-2] and train['cases'][-2]==train['cases'][-3]:
        pred1 = [train['cases'][-1]]*(len(valid))
        pred2 = [train['cases'][-1]]*(len(valid))
        print('......', forecast)
    else:
        model1 = ExponentialSmoothing(np.asarray(train['cases']), trend="add", seasonal="add", seasonal_periods=7)
        model2 = ExponentialSmoothing(np.asarray(train['cases']), trend="add", seasonal="add", seasonal_periods=7, damped=True)
        fit1 = model1.fit()
        pred1 = fit1.forecast(len(valid))
        fit2 = model2.fit()
        pred2 = fit2.forecast(len(valid))
        
    forecast1 = pd.DataFrame(pred1,index = valid.index,columns=['Prediction'])
    rms1 = sqrt(mean_squared_error(valid,forecast1))
    forecast2 = pd.DataFrame(pred2,index = valid.index,columns=['Prediction'])
    rms2 = sqrt(mean_squared_error(valid,forecast2))
    
    print(index, rms1, rms2)
    
    #plot the predictions for validation set
    if do_plot==1:
        plt.plot(train, label='Train')
        plt.plot(valid, label='Valid')
        plt.plot(forecast, label='Prediction')
        plt.legend()
        plt.show()
    return rms1, forecast1, rms2, forecast2

In [11]:
combined1 = []
combined2 = []
columns = confirmed.columns[4:]
for a in range(len(confirmed)):
    df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:], 'date':pd.to_datetime(columns)}).set_index('date')
    #df.index = pd.to_datetime(columns)
    train = df
    valid = df[-7:]
    rms1, forecast1, rms2, forecast2 = my_arima_model(a, train, valid)
    current1 = [rms1, a, valid, forecast1, train]
    combined1.append(current1)
    current2 = [rms2, a, valid, forecast2, train]
    combined2.append(current2)
    

0 16.195297490814255 16.19530587474625
1 1.7111168911288976 1.4887871313634002
2 57.23453952260569 52.45673479091368
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
3 1.7677669529663689 1.7677669529663689
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
4 0.0 0.0
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
5 0.0 0.0
6 62.870913959746794 62.87080533648526
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20          

KeyboardInterrupt: 

In [ ]:
score, y_pred, y_real = evaluate(combined1)
print("Score ",score)
print("Prediction, Actual")
arima_pred_df = pd.DataFrame({"Actual": y_real, "Prediction":y_pred})
#arima_pred_df.to_csv("22_april_cases_holt_pred_df.csv", index=False)
#arima_pred_df.head()

In [ ]:
score, y_pred_7thday, y_pred, y_pred1, y_real  = evaluate_ensemble(combined, combined1)

# Ensembled

In [20]:
ensembled_Results = pd.DataFrame({'7th_yReal': y_real, '7th_yArima': y_pred, '7th_yHolt': y_pred, '7th_yEnsembled': y_pred})
ensembled_Results.to_csv("latest_ensembled_deaths.csv", index=False)